#### Function Calling의 사용
- 이 코드에서 function calling은 모델이 적절한 시점에 외부 함수(get_current_weather)를 호출하도록 유도합니다. 모델이 What is the current weather in {user_location}?와 같은 질문에 응답할 때, 해당 함수가 정의되어 있으면 모델은 그 함수를 호출하여 날씨 정보를 가져오려 합니다.

- function_call="auto" 설정을 통해 GPT 모델은 적절한 함수가 있을 때 자동으로 그 함수를 호출할 수 있습니다. 여기서는 get_current_weather 함수 명세를 통해 사용자가 입력한 위치에 맞는 날씨 정보를 가져올 수 있도록 유도됩니다.

- 만약 모델이 함수 호출을 결정하면, 응답에 function_call이 포함되며, 이때 함수 이름과 그 인자가 함께 반환됩니다. 그런 후에 함수가 호출되고, 결과가 다시 모델에 제공됩니다.

이 과정에서 모델은 단순히 인자를 생성하고, 함수를 호출하는 것은 개발자의 책임입니다. OpenAI의 API는 실제 함수 실행을 수행하지 않으므로, 함수 호출 후 결과를 처리하고 다시 모델에 넘기는 과정을 수동으로 처리하게 됩니다.

In [ ]:
!pip install openai -q

In [17]:
from openai import OpenAI
import requests
import json

client = OpenAI(api_key='')

# 날씨 API 키 설정
weather_api_key = ''

# 날씨정보 호출 함수
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    return data

# 사용자 요청 메세지
messages = [
    {"role":"system", "content":"당신은 친절한 도우미입니다."},
    {"role":"user", "content":"Korea의 Seoul 지역 날씨는 어때?"}
]

# GPT-4 모델호출
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    functions=[{
        "name":"get_current_weather",
        "description":"Get the current weather for a specific location",
        "parameters":{
            "type":"object",
            "properties":{
                "location":{
                    "type":"string",
                    "description":"날씨를 가져올 도시의 이름"
                }
            },
            "required":["location"]
        }
    }],
    function_call="auto"
)

# 모델의 응답메세지를 messages 리스트에 추가하고 출력
response_message = response.choices[0].message
messages.append(response_message)
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Seoul"}', name='get_current_weather'), tool_calls=None)


In [18]:
# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과처리
    if tool_function_name == "get_current_weather":
        location = tool_arguments.get("location")
        weather_results = get_current_weather(location)

        # 함수 호출 결과 메세지 출력
        messages.append({
            "role":"function",
            "name":tool_function_name,
            "content":json.dumps(weather_results) # json 형식으로 반환
        })

        # 모델 재호출
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)

    else:
        print(f"Error : function {tool_function_name} does not exist")
else:
    #도구 호출이 없는경우 결과 반환
    print(response_message.content)

현재 서울의 날씨 정보를 제공할 수 없지만, 일반적으로 서울의 날씨는 계절에 따라 크게 변동이 있습니다. 봄과 가을은 온화하고 기분 좋으며, 여름은 덥고 습하며, 겨울은 춥고 건조합니다. 최신 날씨 정보를 확인하려면 기상청 웹사이트나 날씨 앱을 이용하는 것이 좋습니다.


Q. 도시를 입력받고 외부사이트에서 funcion calling해서 날씨 정보 불러오기

In [36]:
from openai import OpenAI
import requests
import json

client = OpenAI(api_key='')

# 날씨 API 키 설정
weather_api_key = ''

sys_msg = "당신은 도시이름을 입력받으면 해당 도시의 현재 날씨정보를 알려주는 친절한 도우미 입니다."

# 날씨정보 호출 함수
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    return data

while True:
    user_input = input("날씨를 알고 싶은 도시 이름을 입력하세요(종료: exit 입력): ")

    if user_input.lower() == 'exit':
        print("프로그램을 종료합니다.")
        break

    # 사용자 요청 메세지
    messages = [
        {"role":"system", "content": sys_msg},
        {"role":"user", "content": user_input}
    ]

    # GPT-4 모델호출
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        functions=[{
            "name":"get_current_weather",
            "description":"Get the current weather for a specific location",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"날씨를 가져올 도시의 이름"
                    }
                },
                "required":["location"]
            }
        }],
        function_call="auto"
    )

    # 모델의 응답메세지를 messages 리스트에 추가하고 출력
    response_message = response.choices[0].message
    messages.append(response_message)

    # 모델의 응답메세지를 messages 리스트에 추가하고 출력
    response_message = response.choices[0].message
    messages.append(response_message)

    # 도구 호출 여부 확인
    function_call = response_message.function_call
    if function_call:
        tool_function_name = function_call.name
        tool_arguments = json.loads(function_call.arguments)

        # 함수 호출 및 결과처리
        if tool_function_name == "get_current_weather":
            location = tool_arguments.get("location")
            weather_results = get_current_weather(location)

            # 함수 호출 결과 메세지 출력
            messages.append({
                "role":"function",
                "name":tool_function_name,
                "content":json.dumps(weather_results) # json 형식으로 반환
            })

            # 모델 재호출
            model_response_with_function_call = client.chat.completions.create(
                model="gpt-4o-mini-2024-07-18",
                messages=messages
            )
            print(model_response_with_function_call.choices[0].message.content)

        else:
            print(f"Error : function {tool_function_name} does not exist")
    else:
        #도구 호출이 없는경우 결과 반환
        print(response_message.content)

날씨를 알고 싶은 도시 이름을 입력하세요(종료: exit 입력): seoul
현재 서울의 날씨는 다음과 같습니다:

- **온도**: 31.76°C
- **체감 온도**: 38.76°C
- **습도**: 70%
- **바람 속도**: 1.54 m/s (남쪽)
- **구름**: 약간의 구름 (20% 커버리지)
- **기압**: 1009 hPa
- **가시거리**: 10km

따뜻한 날씨에 약간의 구름이 있는 상태입니다. 더운 날씨이니 외출 시 충분한 수분 섭취를 잊지 마세요!
날씨를 알고 싶은 도시 이름을 입력하세요(종료: exit 입력): exit
프로그램을 종료합니다.


## How to call functions with model generated arguments

다음 예제에서는 모델에서 생성된 입력을 갖는 함수를 실행하는 방법을 보여주고 이를 사용하여 데이터베이스에 대한 질문에 답할 수 있는 에이전트를 구현합니다. 단순화를 위해 Chinook 샘플 데이터베이스를 사용하겠습니다 .

참고: 모델이 올바른 SQL을 생성하는 데 완벽하게 신뢰할 수 없기 때문에 프로덕션 환경에서 SQL 생성은 위험할 수 있습니다.

이 코드는 OpenAI의 GPT 모델을 사용하여 SQLite 데이터베이스에서 음악 관련 질문에 대한 답변을 SQL 쿼리로 변환한 후, 해당 쿼리를 실행하여 결과를 반환하는 방식으로 function calling을 구현한 예시입니다. 여기서는 사용자가 앨범 관련 질문을 하면, GPT-4가 질문을 SQL 쿼리로 변환하고, SQLite 데이터베이스를 조회하여 결과를 반환합니다.

In [37]:
import sqlite3
from google.colab import drive

drive.mount('/content/drive')

conn = sqlite3.connect('/content/drive/MyDrive/kdt_240424/m9_LLM/data/Chinook.db')
cursor = conn.cursor()
print('Opened database succesfully')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Opened database succesfully


데이터베이스 테이블 및 열 정보 조회
- get_table_names, get_column_names 함수를 통해 데이터베이스의 테이블 및 열 이름을 가져옵니다.
- 이정보는 나중에 GPT 모델이 SQL 쿼리를 생성할 때 사용할 스키마정보를 제공하는데 활용됩니다.

In [38]:
# 데이버베이스에서 테이블 목록ㅊ을 추출하는 함수. sqlite_master 테이블에서 type이 table인 항목들의 이름을 가져옴
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

# PRAGMA table_info(table_name) 명령을 사용하여 테이블의 스키마 정보를 가져오고, 컬럼 이름을 리스트로 반환
def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


데이터베이스의 테이블과 열 정보를 문자열 형태로 저장하여, 나중에 SQL 쿼리 작성시 참조할 수 있도록 함.

In [39]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

tools라는 리스트에 ask_database라는 함수 명세를 정의합니다.
- 이 함수는 사용자의 질문에 맞는 SQL 쿼리를 받아 데이터베이스에서 정보를 조회하고, 이를 반환하는 기능을 합니다.
- 함수의 매개변수로 query가 있으며, 이는 SQL 쿼리를 텍스트 형태로 전달받아 실행하는 구조입니다.
- 중요한 점은 database_schema_string이 함수 설명에 포함되어 있어, 모델이 데이터베이스 스키마에 맞는 SQL 쿼리를 생성할 수 있도록 도움을 줍니다.

In [52]:
tools = [
    {
    "type": "function",
    "function": {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description" : f"""
                    SQL query extracting info to answer the user's question.
                    SQL should be written using this database schema:
                    {database_schema_string}
                    The query should be returned in plain text, not in JSON.
                    """,
                }
            },
            "required": ["query"]
        }
    }
}]

##### Steps to invoke a function call using Chat Completions API:

- 1단계 : 모델이 사용할 도구를 선택하도록 하는 내용으로 모델을 프롬프트합니다. 함수 이름 및 서명과 같은 도구에 대한 설명은 '도구' 목록에 정의되어 API 호출에서 모델에 전달됩니다. 선택한 경우 함수 이름과 매개변수가 응답에 포함됩니다.
- 2단계 : 모델이 함수를 호출하려고 하는지 프로그래밍적으로 확인합니다. 참이면 3단계로 진행합니다.
- 3단계 : 응답에서 함수 이름과 매개변수를 추출하고 매개변수와 함께 함수를 호출합니다. 결과를 메시지에 추가합니다.
- 4단계 : 메시지 목록으로 채팅 완료 API를 호출하여 응답을 가져옵니다.

In [53]:
# 사용자의 요청 메세지 정의
messages = [{
    "role":"user",
    "content": "가장 많은 트랙을 가진 앨범이 뭐야?"
}]

# 사용자의 지룬에 대한 응답을 생성. tools와 tool_choice 파라미터는 모델이 데이터베이스 쿼리를 인식하고 자동으로 도구를 선택할 수 있도록 돕습니다.
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    tools=tools,
    tool_choice="auto" # 모델이 자동으로 함수 호출을 결정, SQL 쿼리로 변환할 수 있다고 판단하면 자동으로 ask_database 함수를 호출하게 됩니다.
)

# 모델의 응답메세지를 messages 리스트에 추가하고 출력
response_message = response.choices[0].message
messages.append(response_message)
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xM4iyzNmaZVOdA6vcA7Gx2eq', function=Function(arguments='{"query":"SELECT Album.Title, COUNT(Track.TrackId) AS TrackCount \\nFROM Album \\nJOIN Track ON Album.AlbumId = Track.AlbumId \\nGROUP BY Album.AlbumId \\nORDER BY TrackCount DESC \\nLIMIT 1;"}', name='ask_database'), type='function')])


In [54]:
# 모델 응답에서 도구 호출이 포함되어 있는지 확인하고, 도구 호출이 있다면 도구호출 ID, 함수 이름 및 쿼리 문자열을 추출
tool_call = response_message.tool_calls
if tool_call:
    # if true the model will return the name of the tool / function to call and the argument(s)
    tool_call_id = tool_call[0].id
    tool_function_name = tool_call[0].function.name
    tool_query_string = json.loads(tool_call[0].function.arguments)['query']

    # 도구 호출 함수 이름이 'ask_database'인 경우, ask_database 함수를 호출하여 데이터베이스 쿼리를 실행하고 결과를 messages 리스트에 추가.
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call_id,
            "name": tool_function_name,
            "content": results
        })

        # 도구 호출 결과가 포함된 messsages 리스트를 사용하여 모델을 다시 호출하고 최종 응답을 출력
        model_response_with_fuctions_call  = client.chat_completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages
        )
        print(model_response_with_fuctions_call.choices[0].message.content)
        # 도구 호출 함수 이름이 'ask_database'가 아닌 경우 오류 메세지를 출력하거나, 도구 호출이 없음녀 모델의 응답내용을 출력
    else:
        print(f"Error : function {tool_function_name} does not exist")
else:
    print(response_message.content)

AttributeError: 'OpenAI' object has no attribute 'chat_completions'

## LangChain
- 자연어 처리(NLP)와 생성형 AI 응용 프로그램을 개발하기 위한 프레임워크입니다. 주로 대형 언어 모델(LLMs)과 같은 최신 NLP 기술을 기반으로 하여 다양한 작업을 자동화하거나 개선할 수 있는 도구와 서비스를 제공합니다.
- 이 프레임워크는 특히 언어 모델의 기능을 확장하고 이를 보다 쉽게 사용할 수 있도록 설계되었습니다.
- LangChain의 핵심 목표는 언어 모델을 활용하여 여러 복잡한 작업을 수행할 수 있도록 돕는 것이며, 특히 긴 텍스트, 문서 체인 또는 여러 단계의 워크플로를 필요로 하는 복잡한 응용 프로그램에 유용합니다.

### LangChain의 구성 요소:
- Language Models: 언어 모델 자체를 사용하여 텍스트 생성, 요약, 번역 등의 작업을 수행합니다.
- Chains: 여러 모델 호출을 연결하여 복잡한 작업을 수행하는 논리적 흐름을 정의할 수 있습니다. 예를 들어, 텍스트를 요약한 후 요약된 텍스트에 대한 질의응답을 수행하는 체인을 만들 수 있습니다.
- Agents: 주어진 작업에 따라 다양한 툴을 선택하고 사용할 수 있는 지능형 에이전트입니다. 예를 들어, 정보 검색, API 호출 등을 수행하는 역할을 합니다.
- Memory: 이전의 상호작용 또는 맥락을 기억하는 기능입니다. 이를 통해 대화형 AI나 컨텍스트를 유지해야 하는 애플리케이션을 구현할 수 있습니다.

### 활용 사례:
- 대화형 에이전트: 사용자와의 대화에서 맥락을 유지하며 대답할 수 있는 챗봇 개발.
문서 처리: 긴 문서나 여러 문서의 내용을 요약하거나 분석하는 애플리케이션.
- 지식 탐색: 사용자가 특정 주제에 대해 질문하면, 관련된 정보를 검색하고 이를 바탕으로 대답을 제공하는 시스템.

### 통합:
- LangChain은 다양한 데이터 소스, API 및 언어 모델과 통합될 수 있으며, 이를 통해 다양한 도메인에서 사용될 수 있습니다. 예를 들어, SQL 데이터베이스에 쿼리를 보내고 결과를 요약하거나, 웹에서 정보를 수집한 후 이를 기반으로 질문에 답하는 작업을 수행할 수 있습니다.

이 프레임워크는 특히 연구자, 데이터 사이언티스트, 개발자들이 생성형 AI를 활용하여 복잡한 텍스트 기반 응용 프로그램을 구축하는 데 큰 도움이 됩니다. LangChain은 이러한 작업을 더 쉽게, 더 직관적으로 구현할 수 있게 도와줍니다.

# Lang Chain 패키지를 사용하여 RAG 시스템을 구성하는 예제
주어진 질문에 대해 텍스트 파일에 저장된 문서에서 관련 정보를 검색하고, 이를 기반으로 GPT 모델이 답변을 생성하는 것.
- langchain-openai: LangChain과 OpenAI를 연결하는 패키지.
- faiss-cpu: 벡터 검색을 위한 FAISS 라이브러리의 CPU 버전.
- langchain_community: LangChain의 커뮤니티에서 개발된 추가 도구.
- tiktoken: OpenAI 모델에서 사용하는 토큰화 도구.

In [55]:
!pip install -q openai
!pip install -q langchain-openai
!pip install -q faiss-cpu
!pip install -q langchain_community
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00


In [56]:
from openai import OpenAI

model = 'gpt-4o-mini-2024-07-18'
client = OpenAI(api_key='')

In [57]:
content = """
Document 1:
Polar bears are directly impacted by climate change due to their dependence on sea ice for hunting seals. As global temperatures rise, sea ice is melting earlier and forming later each year, reducing the time polar bears have to hunt. This has led to decreased body condition, lower cub survival rates, and in some cases, increased mortality.

Document 2:
The loss of sea ice habitat is one of the most significant threats to polar bear populations. As the ice retreats, polar bears are forced to travel greater distances to find food, leading to increased energy expenditure. This can result in malnutrition and a decline in reproductive success, further endangering the species.

Document 3:
Climate change is not only affecting polar bears' hunting grounds but also their denning areas. Warmer temperatures and unstable snow conditions are making it more difficult for pregnant females to find suitable denning sites, which is crucial for the birth and survival of their cubs. This adds an additional layer of risk to polar bear populations already struggling due to loss of sea ice.

Document 4:
As their traditional food sources become less accessible, some polar bears have been observed scavenging on human waste or preying on seabirds and their eggs. While this behavior may provide some short-term relief, it does not replace the high-fat diet they obtain from seals, which is essential for their long-term survival in the harsh Arctic environment.

Document 5:
Recent studies suggest that if current trends in greenhouse gas emissions continue, polar bears could face extinction within this century. Conservation efforts are focusing on reducing global emissions and protecting critical polar bear habitats, but these efforts may not be sufficient if the climate continues to warm at the current rate.
"""

with open('document.txt', 'w') as file:
    file.write(content)

문서 1:
북극곰은 물개 사냥을 위해 해빙에 의존하기 때문에 기후 변화에 직접적인 영향을 받습니다. 지구의 온도가 상승함에 따라 매년 해빙이 더 일찍 녹고 늦게 형성되어 북극곰이 사냥해야 하는 시간이 줄어듭니다. 이로 인해 신체 상태가 감소하고 새끼 생존율이 낮아지며 일부 경우 사망률이 증가했습니다.

문서 2:
해빙 서식지의 손실은 북극곰 개체수에 대한 가장 심각한 위협 중 하나입니다. 얼음이 줄어들면서 북극곰은 먹이를 찾기 위해 더 먼 거리를 이동해야 하므로 에너지 소비가 증가합니다. 이로 인해 영양실조가 발생하고 번식 성공률이 감소하여 종을 더욱 위험에 빠뜨릴 수 있습니다.

문서 3:
기후 변화는 북극곰의 사냥터뿐만 아니라 굴 지역에도 영향을 미칩니다. 따뜻한 기온과 불안정한 눈 상태로 인해 임신한 암컷이 적절한 굴을 찾는 것이 더 어려워지고 있으며, 이는 새끼의 출산과 생존에 매우 중요합니다. 이는 이미 해빙 손실로 인해 어려움을 겪고 있는 북극곰 개체수에 추가적인 위험 요소를 추가합니다.

문서 4:
전통적인 식량원에 대한 접근이 어려워짐에 따라 일부 북극곰은 인간의 배설물을 청소하거나 바닷새와 알을 잡아먹는 것이 관찰되었습니다. 이러한 행동이 단기적으로는 어느 정도 도움이 될 수 있지만, 가혹한 북극 환경에서 장기적으로 생존하는 데 필수적인 물개에게서 얻는 고지방 식단을 대체할 수는 없습니다.

문서 5:
최근 연구에 따르면 현재의 온실가스 배출 추세가 계속된다면 북극곰은 금세기 안에 멸종에 직면할 수 있다고 합니다. 보존 노력은 전 세계 탄소 배출량을 줄이고 중요한 북극곰 서식지를 보호하는 데 초점을 맞추고 있지만, 기후가 현재 속도로 계속 따뜻해지면 이러한 노력만으로는 충분하지 않을 수 있습니다.